In [ ]:
import sys
sys.path.append("../")

In [ ]:
import os

import numpy as np
import pandas as pd
import random

from agents.matchmanager import buildMatchManager

from datetime import datetime
from multiprocessing import Pool

In [ ]:
agents = [
    'RandomAgent', 
#     'AlphaBetaAgent', 
    'GreedyAgent'
]
scenarios = [
    'scenarioTest1v1', 
    'scenarioTest2v2', 
    'scenarioJunctionExo', 
    'scenarioJunction', 
#     'scenarioRoadblock', 
#     'scenarioBridgeHead', 
#     'scenarioCrossingTheCity'
]

In [ ]:
def game(args):
    scenario, red, blue, seed = args
    vectors=[]

    output = f'../data/{scenario}.{red}.{blue}.{seed}.pkl.gz'
    
    if os.path.exists(output):
        return
    
    mm = buildMatchManager('', scenario, red, blue, seed=seed)
    
    while not mm.end:
        mm.nextStep()
        v = list(mm.state.vector())
        #mm.states_history in lista di vettori
        #mm.actions_history in lista di vettori

        vectors.append(v)
    
    for v in vectors:
        v += [mm.winner, scenario, red, blue, seed]
    
    cols = list(mm.state.vectorInfo()) + ['winner', 'meta_scenario', 'meta_p_red', 'meta_p_blue', 'meta_seed']
        
    df = pd.DataFrame(data=vectors, columns=cols)
    df.to_pickle(output, compression='gzip')


In [ ]:
args = []

np.random.seed(20201109)

for _ in range(1000000):
    scenario = np.random.choice(scenarios)
    red = np.random.choice(agents)
    blue = np.random.choice(agents)
    seed = np.random.randint(100000000, 999999999)
    
    args.append((scenario, red, blue, seed))

In [ ]:
with Pool(64) as p:
    p.map(game, args)

In [ ]:
def compress(scenario):
    today = datetime.today().strftime('%Y-%m-%d')
    data_dir = '../data'
    
    files = [f for f in os.listdir(data_dir) if scenario in f]
    
    dfs = [
        pd.concat([
            pd.read_pickle(os.path.join(data_dir, f), compression='gzip') for f in files[i:i+1000]
        ]) for i in range(0, len(files), 1000)
    ]
    
    dfs = pd.concat(dfs)
    
    dfs.to_pickle(f'../data.{today}.{scenario}.pkl.gz', compression='gzip')

In [ ]:
with Pool(len(scenarios)) as p:
    p.map(compress, scenarios)

In [ ]:
df = pd.read_pickle('../data.scenarioJunction.pkl.gz', compression='gzip')

In [ ]:
len(sorted(list(df.columns)))

In [ ]:
for c in df.columns:
    print(c)